In [1]:
import numpy as np
import pandas as pd

import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans

import lightgbm as lgb
import xgboost as xgb
import missingno as msno
from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_curve, auc

In [2]:
import shap

KeyboardInterrupt: 

In [3]:
pd.options.display.max_rows = 222
pd.set_option("display.max_columns", 50)

In [4]:
df_train = pd.read_csv(
    "../geekbrains-competitive-data-analysis/train.csv"
)

df_test = pd.read_csv(
    "../geekbrains-competitive-data-analysis/test.csv"
)

print("gf_train.shape = {} rows, {} cols".format(*df_train.shape))
print("df_test.shape = {} rows, {} cols".format(*df_test.shape))

gf_train.shape = 110093 rows, 3 cols
df_test.shape = 165141 rows, 2 cols


In [5]:
#df_train = create_dummie(df_train)
#df_test = create_dummie(df_test)

df_train['NAME_CONTRACT_TYPE'] = (df_train['NAME_CONTRACT_TYPE'] == 'Cash').astype(int)
df_test['NAME_CONTRACT_TYPE'] = (df_test['NAME_CONTRACT_TYPE'] == 'Cash').astype(int)

In [6]:
df_sample_submit = pd.read_csv(
    "../geekbrains-competitive-data-analysis/sample_submit.csv"
)

print("sample_submit.shape = {} rows, {} cols".format(*df_sample_submit.shape))
df_sample_submit.head()

sample_submit.shape = 165141 rows, 2 cols


,APPLICATION_NUMBER,TARGET
0,123724268,0
1,123456549,0
2,123428178,0
3,123619984,0
4,123671104,0


# функции

In [7]:
def create_freq_feature(data: pd.DataFrame,
                        feature: str
                       ) -> pd.DataFrame:
    freq = data[feature].value_counts()
    data[feature] = data[feature].map(freq).astype('float')
    data[feature] = data[feature].fillna(0.0).astype('float')
    data[feature] = data[feature] / data.shape[0]
    
    return data

In [8]:
def numeric_feature_preprocessing(data: pd.DataFrame,
                         numeric_feature=list()) -> pd.DataFrame:
    
    if len(numeric_feature) == 0:
        numeric_feature = data.select_dtypes(include=[np.number]).columns
    
    for feature in numeric_feature:
        max_ = data[feature].quantile(.99)
        min_ = data[feature].quantile(.01)
        data.loc[data[feature] > max_, [feature]] = max_
        data.loc[data[feature] < min_, [feature]] = min_
        
    return data

In [9]:
def create_dummie(data: pd.DataFrame) -> pd.DataFrame:
    """
    

    """
    numeric_feature = data.select_dtypes(include=[np.number]).columns

    сategorical_feature = list(set(data.columns) - set(numeric_feature))

    data_copy = data.copy()

    for i in range(len(сategorical_feature)):
        print(f'feature {сategorical_feature[i]}')
        сategorical_values = data_copy[сategorical_feature[i]].unique()
        print(f'сategorical values {сategorical_values}')

        for cat in сategorical_values:
            dummie_feature_name = f'{сategorical_feature[i]}_{cat}'

            if str(cat) != 'nan':
                print(f'    dummie feature name: {dummie_feature_name}')
                data_copy[f'{сategorical_feature[i]}_{cat}'] = 0
                data_copy.loc[data_copy.loc[:,сategorical_feature[i]]==cat, 
                                                 [dummie_feature_name]] = 1      

        data_copy.drop([сategorical_feature[i]], axis='columns', inplace=True)

    print(f"count of data {data_copy.shape}")
    return data_copy

# история предыдущих заявок клиента.
df_applications_history

In [10]:
#!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
# история предыдущих заявок клиента.
df_applications_history = pd.read_csv(
    "../geekbrains-competitive-data-analysis/applications_history.csv"
)

print("applications_history.shape = {} rows, {} cols".format(*df_applications_history.shape))
df_applications_history.head()

applications_history.shape = 1670214 rows, 26 cols


,PREV_APPLICATION_NUMBER,APPLICATION_NUMBER,NAME_CONTRACT_TYPE,AMOUNT_ANNUITY,AMT_APPLICATION,AMOUNT_CREDIT,AMOUNT_PAYMENT,AMOUNT_GOODS_PAYMENT,NAME_CONTRACT_STATUS,DAYS_DECISION,NAME_PAYMENT_TYPE,CODE_REJECT_REASON,NAME_TYPE_SUITE,NAME_CLIENT_TYPE,NAME_GOODS_CATEGORY,NAME_PORTFOLIO,NAME_PRODUCT_TYPE,SELLERPLACE_AREA,CNT_PAYMENT,NAME_YIELD_GROUP,DAYS_FIRST_DRAWING,DAYS_FIRST_DUE,DAYS_LAST_DUE_1ST_VERSION,DAYS_LAST_DUE,DAYS_TERMINATION,NFLAG_INSURED_ON_APPROVAL
0,49298709,123595216,NaN,1730.430,17145.0,17145.0,0.0,17145.0,Approved,73,Cash through the bank,XAP,NaN,Repeater,Mobile,POS,XNA,35,12.0,middle,365243.0,42.0,300.0,42.0,37.0,0.0
1,50070639,123431468,Cash,25188.615,607500.0,679671.0,NaN,607500.0,Approved,164,XNA,XAP,Unaccompanied,Repeater,XNA,Cash,x-sell,-1,36.0,low_action,365243.0,134.0,916.0,365243.0,365243.0,1.0
2,49791680,123445379,Cash,15060.735,112500.0,136444.5,NaN,112500.0,Approved,301,Cash through the bank,XAP,"Spouse, partner",Repeater,XNA,Cash,x-sell,-1,12.0,high,365243.0,271.0,59.0,365243.0,365243.0,1.0
3,50087457,123499497,Cash,47041.335,450000.0,470790.0,NaN,450000.0,Approved,512,Cash through the bank,XAP,NaN,Repeater,XNA,Cash,x-sell,-1,12.0,middle,365243.0,482.0,152.0,182.0,177.0,1.0
4,49052479,123525393,Cash,31924.395,337500.0,404055.0,NaN,337500.0,Refused,781,Cash through the bank,HC,NaN,Repeater,XNA,Cash,walk-in,-1,24.0,high,NaN,NaN,NaN,NaN,NaN,NaN


In [11]:
msno.matrix(df_applications_history)

KeyboardInterrupt: 

In [ ]:
numerical_features_df_applications_history = df_applications_history.select_dtypes(include=[np.number]).columns
numerical_features_df_applications_history = numerical_features_df_applications_history.drop('APPLICATION_NUMBER')
numerical_features_df_applications_history = numerical_features_df_applications_history.drop('PREV_APPLICATION_NUMBER')
numerical_features_df_applications_history

NAME_CONTRACT_TYPE

In [ ]:
df_applications_history['NAME_CONTRACT_TYPE'].value_counts()

In [ ]:
df_applications_history['NAME_CONTRACT_TYPE'] = (df_applications_history['NAME_CONTRACT_TYPE'] == 'Cash').astype(int)

AMOUNT_ANNUITY, AMOUNT_GOODS_PAYMENT

In [ ]:
df_applications_history = df_applications_history.fillna(value=
                                                         {'AMOUNT_ANNUITY': 0, 
                                                          'AMOUNT_GOODS_PAYMENT': 0})

AMOUNT_PAYMENT

In [ ]:
df_applications_history.drop(['AMOUNT_PAYMENT'], axis='columns', inplace=True)
numerical_features_df_applications_history = numerical_features_df_applications_history.drop('AMOUNT_PAYMENT')

NAME_TYPE_SUITE

In [ ]:
df_applications_history['NAME_TYPE_SUITE'].value_counts()

In [ ]:
df_applications_history = create_freq_feature(df_applications_history, 'NAME_TYPE_SUITE')

NAME_CONTRACT_STATUS

In [ ]:
df_applications_history['NAME_CONTRACT_STATUS'].value_counts()

In [ ]:
freq = pd.Series(data=[1, 0, 0.5, 0.5], index=['Approved', 'Canceled', 'Refused', 'Unused offer'], dtype='float')
df_applications_history['NAME_CONTRACT_STATUS'] = df_applications_history['NAME_CONTRACT_STATUS'].map(freq).astype('float')

NAME_PAYMENT_TYPE

In [ ]:
df_applications_history['NAME_PAYMENT_TYPE'].value_counts()

In [ ]:
df_applications_history = create_freq_feature(df_applications_history, 'NAME_PAYMENT_TYPE')

CODE_REJECT_REASON

In [ ]:
df_applications_history['CODE_REJECT_REASON'].value_counts()

In [ ]:
df_applications_history = create_freq_feature(df_applications_history, 'CODE_REJECT_REASON')

NAME_CLIENT_TYPE

In [ ]:
df_applications_history['NAME_CLIENT_TYPE'].value_counts()

In [ ]:
df_applications_history = create_freq_feature(df_applications_history, 'NAME_CLIENT_TYPE')

NAME_GOODS_CATEGORY

In [ ]:
df_applications_history['NAME_GOODS_CATEGORY'].value_counts()

In [ ]:
df_applications_history = create_freq_feature(df_applications_history, 'NAME_GOODS_CATEGORY')

NAME_PORTFOLIO, NAME_PRODUCT_TYPE, NAME_YIELD_GROUP

In [ ]:
df_applications_history = create_freq_feature(df_applications_history, 'NAME_PORTFOLIO')
df_applications_history = create_freq_feature(df_applications_history, 'NAME_PRODUCT_TYPE')
df_applications_history = create_freq_feature(df_applications_history, 'NAME_YIELD_GROUP')

Other

In [ ]:
df_applications_history = df_applications_history.fillna(0)

numeric

In [ ]:
# df_applications_history = numeric_feature_preprocessing(df_applications_history, 
#                                                         numeric_feature=numerical_features_df_applications_history)

visual

In [ ]:
standard_scaler = StandardScaler()
df_applications_history_scale = \
            pd.DataFrame(standard_scaler.fit_transform(df_applications_history), 
                         columns=df_applications_history.columns)

In [ ]:
from sklearn.manifold import TSNE
tsne = TSNE(n_components=3, learning_rate=250, random_state=42)

In [ ]:
df_applications_history_scale.drop(['PREV_APPLICATION_NUMBER', 'APPLICATION_NUMBER'], axis='columns', inplace=True)

In [ ]:
%%time

applications_history_tsne = tsne.fit_transform(df_applications_history_scale.sample(2000))

In [ ]:
from mpl_toolkits.mplot3d import Axes3D
plt.rcParams['figure.figsize'] = [10, 10]

fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')
ax.scatter(applications_history_tsne[:, 0],
        applications_history_tsne[:, 1],
        applications_history_tsne[:, 2])

## model

In [ ]:
%%time
df_train_applications_history = df_applications_history.copy()

df_train_applications_history = pd.pivot_table(df_train_applications_history,  
                                                index=['APPLICATION_NUMBER'], aggfunc=np.max)


df_train_applications_history = df_train.loc[:, ['APPLICATION_NUMBER', 'TARGET']].merge(
                                               df_train_applications_history, 
                                               left_on='APPLICATION_NUMBER', 
                                               right_on='APPLICATION_NUMBER', 
                                               how='inner')

cor = df_train_applications_history.corr()

# df_train_applications_history, df_valid_applications_history = train_test_split(df_train_applications_history, 
#                                                                                 test_size=0.3,  
#                                                                                 random_state=42)

y_train_applications_history = df_train_applications_history["TARGET"]
# y_valid_applications_history = df_valid_applications_history["TARGET"]

df_train_applications_history.drop(["TARGET", 'APPLICATION_NUMBER', 'PREV_APPLICATION_NUMBER'], axis='columns', inplace=True)
# df_valid_applications_history.drop(["TARGET",'APPLICATION_NUMBER', 'PREV_APPLICATION_NUMBER'], axis='columns', inplace=True)

print("df_train_applications_history.shape = {} rows, {} cols".format(*df_train_applications_history.shape))
# print("df_valid_applications_history.shape = {} rows, {} cols".format(*df_valid_applications_history.shape))

dtrain = lgb.Dataset(data=df_train_applications_history, label=y_train_applications_history)
# dvalid = lgb.Dataset(data=df_valid_applications_history, label=y_valid_applications_history)

params = {
    "boosting_type": "goss",
    "objective": "binary",
    "metric": "auc",
    "learning_rate": 0.01,
    "n_estimators": 2000,
    "n_jobs": 6,
    "seed": 20
}

model_lgb = lgb.train(
    params=params,
    train_set=dtrain,
    num_boost_round=1000,
    valid_sets=[dtrain],
    early_stopping_rounds=50,
    verbose_eval=500
)

In [ ]:
feature_importances = pd.DataFrame(model_lgb.feature_importance().reshape(-1,len(model_lgb.feature_importance())), columns=model_lgb.feature_name(), index=['importances'])
feature_high_importances = feature_importances.loc[:, 
                                                   feature_importances.loc['importances', :] > feature_importances.loc['importances', :].mean()
                                                  ].columns
feature_importances

In [ ]:
df_train_applications_history["TARGET"] = y_train_applications_history

df_train_applications_history, df_valid_applications_history = train_test_split(df_train_applications_history, 
                                                                                test_size=0.3,  
                                                                                random_state=42)

y_train_applications_history = df_train_applications_history["TARGET"]
y_valid_applications_history = df_valid_applications_history["TARGET"]

df_train_applications_history.drop(["TARGET"], axis='columns', inplace=True)
df_valid_applications_history.drop(["TARGET"], axis='columns', inplace=True)

print("df_train_applications_history.shape = {} rows, {} cols".format(*df_train_applications_history.shape))
print("df_valid_applications_history.shape = {} rows, {} cols".format(*df_valid_applications_history.shape))

dtrain = lgb.Dataset(data=df_train_applications_history, label=y_train_applications_history)
dvalid = lgb.Dataset(data=df_valid_applications_history, label=y_valid_applications_history)

dtrain = lgb.Dataset(data=df_train_applications_history[feature_high_importances], label=y_train_applications_history)
dvalid = lgb.Dataset(data=df_valid_applications_history[feature_high_importances], label=y_valid_applications_history)

params = {
    "boosting_type": "goss",
    "objective": "binary",
    "metric": "auc",
    "learning_rate": 0.01,
    "n_estimators": 50000,
    "n_jobs": 6,
    "seed": 20
}

model_lgb = lgb.train(
    params=params,
    train_set=dtrain,
    num_boost_round=1000,
    valid_sets=[ dtrain, dvalid ],
    early_stopping_rounds=1000,
    verbose_eval=500
)

result

In [ ]:
kmeans_n_clusters_df_applications_history = 10

kmeans = KMeans(n_clusters=kmeans_n_clusters_df_applications_history,
                max_iter=10000, random_state=1)

In [ ]:
%%time

kmeans_train = kmeans.fit_predict(df_applications_history_scale[feature_high_importances])

In [ ]:
df_applications_history_result = pd.DataFrame(pd.get_dummies(kmeans_train, prefix='applications_history'))
df_applications_history_result_columns = df_applications_history_result.columns
df_applications_history_result['APPLICATION_NUMBER'] = df_applications_history['APPLICATION_NUMBER']

In [ ]:
df_applications_history_result = pd.pivot_table(df_applications_history_result, 
                                                values=df_applications_history_result_columns, 
                                                index=['APPLICATION_NUMBER'], aggfunc=np.sum)

In [ ]:
df_applications_history_result['sum'] = df_applications_history_result.sum(axis=1)
for col in df_applications_history_result_columns:
    df_applications_history_result[col] = df_applications_history_result[col] / df_applications_history_result['sum']
df_applications_history_result.drop(['sum'], axis='columns', inplace=True)  

In [ ]:
# fig, axes = plt.subplots(figsize = (25,25))
# plt.title("Correlation of Numeric Features with Sale Price", y=1, size=16)
# sns.heatmap(cor, square=True, vmax=1, cmap="viridis")

In [ ]:
df_train_applications_history = df_applications_history.copy()

df_train_applications_history = pd.pivot_table(df_train_applications_history,  
                                                index=['APPLICATION_NUMBER'], aggfunc=np.max)

df_applications_history_result = df_train_applications_history[feature_high_importances]
#df_applications_history_result['APPLICATION_NUMBER'] = df_train_applications_history['APPLICATION_NUMBER']

# данные БКИ о предыдущих кредитах клиента;
df_bki

In [ ]:
#!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
# данные БКИ о предыдущих кредитах клиента;
df_bki = pd.read_csv(
    "../geekbrains-competitive-data-analysis/bki.csv"
)

print("bki.shape = {} rows, {} cols".format(*df_bki.shape))

In [ ]:
df_bki.info()

In [ ]:
msno.matrix(df_bki)

In [ ]:
numerical_features_df_bki = df_bki.select_dtypes(include=[np.number]).columns
numerical_features_df_bki = numerical_features_df_bki.drop('APPLICATION_NUMBER')

numerical_features_df_bki

CREDIT_ACTIVE, CREDIT_CURRENCY, CREDIT_TYPE

In [ ]:
df_bki['CREDIT_ACTIVE'].value_counts()

In [ ]:
df_bki['CREDIT_CURRENCY'].value_counts()

In [ ]:
df_bki['CREDIT_TYPE'].value_counts()

In [ ]:
df_bki = create_freq_feature(df_bki, 'CREDIT_ACTIVE')
df_bki = create_freq_feature(df_bki, 'CREDIT_CURRENCY')
df_bki = create_freq_feature(df_bki, 'CREDIT_TYPE')

DAYS_CREDIT_ENDDATE

In [ ]:
mean = df_bki.loc[df_bki['DAYS_CREDIT_ENDDATE'].notnull(), ['DAYS_CREDIT_ENDDATE']].mean()
df_bki['DAYS_CREDIT_ENDDATE'] = df_bki['DAYS_CREDIT_ENDDATE'].fillna(mean[0])

DAYS_ENDDATE_FACT

In [ ]:
df_bki.loc[df_bki['DAYS_ENDDATE_FACT'].isnull(), ['DAYS_ENDDATE_FACT']] = df_bki['DAYS_CREDIT_ENDDATE']

AMT_CREDIT_MAX_OVERDUE

In [ ]:
df_bki['AMT_CREDIT_MAX_OVERDUE'] = df_bki['AMT_CREDIT_MAX_OVERDUE'].fillna(0)

other

In [ ]:
df_bki.drop(['AMT_ANNUITY'], axis='columns', inplace=True)
numerical_features_df_bki = numerical_features_df_bki.drop('AMT_ANNUITY')

In [ ]:
df_bki = df_bki.fillna(0)

num

In [ ]:
# df_bki = numeric_feature_preprocessing(df_bki, 
#                                        numeric_feature=numerical_features_df_bki)

select feature

In [ ]:
%%time
df_train_bki = df_bki.copy()

df_train_bki = pd.pivot_table(df_train_bki,  
                              index=['APPLICATION_NUMBER'], aggfunc=np.max)


df_train_bki = df_train.loc[:, ['APPLICATION_NUMBER', 'TARGET']].merge(
                           df_train_bki, 
                           left_on='APPLICATION_NUMBER', 
                           right_on='APPLICATION_NUMBER', 
                           how='inner')

cor = df_train_bki.corr()

# df_train_bki, df_valid_bki = train_test_split(df_train_bki, 
#                                               test_size=0.3,  
#                                               random_state=42)

y_train_bki = df_train_bki["TARGET"]
# y_valid_bki = df_valid_bki["TARGET"]

df_train_bki.drop(["TARGET", 'APPLICATION_NUMBER', 'BUREAU_ID'], axis='columns', inplace=True)
# df_valid_bki.drop(["TARGET",'APPLICATION_NUMBER'], axis='columns', inplace=True)

print("df_train_bki.shape = {} rows, {} cols".format(*df_train_bki.shape))
# print("df_valid_bki.shape = {} rows, {} cols".format(*df_valid_bki.shape))

dtrain = lgb.Dataset(data=df_train_bki, label=y_train_bki)
# dvalid = lgb.Dataset(data=df_valid_bki, label=y_valid_bki)

params = {
    "boosting_type": "goss",
    "objective": "binary",
    "metric": "auc",
    "learning_rate": 0.01,
    "n_estimators": 5000,
    "n_jobs": 6,
    "seed": 20
}

model_lgb = lgb.train(
    params=params,
    train_set=dtrain,
    num_boost_round=1000,
    valid_sets=[dtrain],
    early_stopping_rounds=50,
    verbose_eval=500
)

feature_importances = pd.DataFrame(model_lgb.feature_importance().reshape(-1,len(model_lgb.feature_importance())), columns=model_lgb.feature_name(), index=['importances'])
feature_high_importances = feature_importances.loc[:, 
                                                   feature_importances.loc['importances', :] > feature_importances.loc['importances', :].mean()
                                                  ].columns
print(feature_importances)


df_train_bki["TARGET"] = y_train_bki

df_train_bki, df_valid_bki = train_test_split(df_train_bki, 
                                              test_size=0.3,  
                                              random_state=42)

y_train_bki = df_train_bki["TARGET"]
y_valid_bki = df_valid_bki["TARGET"]

df_train_bki.drop(["TARGET"], axis='columns', inplace=True)
df_valid_bki.drop(["TARGET"], axis='columns', inplace=True)

print("df_train_bki.shape = {} rows, {} cols".format(*df_train_bki.shape))
print("df_valid_bki.shape = {} rows, {} cols".format(*df_valid_bki.shape))

dtrain = lgb.Dataset(data=df_train_bki[feature_high_importances], label=y_train_bki)
dvalid = lgb.Dataset(data=df_valid_bki[feature_high_importances], label=y_valid_bki)

params = {
    "boosting_type": "goss",
    "objective": "binary",
    "metric": "auc",
    "learning_rate": 0.01,
    "n_estimators": 50000,
    "n_jobs": 6,
    "seed": 20
}

model_lgb = lgb.train(
    params=params,
    train_set=dtrain,
    num_boost_round=1000,
    valid_sets=[ dtrain, dvalid ],
    early_stopping_rounds=1000,
    verbose_eval=500
)

scale

In [ ]:
# df_bki_scale = pd.DataFrame(standard_scaler.fit_transform(df_bki), 
#                columns=df_bki.columns)

In [ ]:
# df_bki_scale.drop(['APPLICATION_NUMBER', 'BUREAU_ID'], axis='columns', inplace=True)

In [ ]:
#kmeans_bki = KMeans(n_clusters=10, max_iter=10000, random_state=1)

In [ ]:
%%time

#kmeans_train_bki = kmeans_bki.fit_predict(df_bki_scale[feature_high_importances])

In [ ]:
# df_bki_result = pd.DataFrame(pd.get_dummies(kmeans_train_bki, prefix='bki'))
# df_bki_result_columns = df_bki_result.columns
# df_bki_result['APPLICATION_NUMBER'] = df_bki['APPLICATION_NUMBER']

In [ ]:
# df_bki_result = pd.pivot_table(df_bki_result, values=df_bki_result_columns, index=['APPLICATION_NUMBER'], aggfunc=np.sum)

In [ ]:
# df_bki_result['sum'] = df_bki_result.sum(axis=1)
# for col in df_bki_result_columns:
#     df_bki_result[col] = df_bki_result[col] / df_bki_result['sum']
# df_bki_result.drop(['sum'], axis='columns', inplace=True)  

In [ ]:
df_bki_result = df_bki.copy()

df_bki_result = pd.pivot_table(df_bki_result,  
                               index=['APPLICATION_NUMBER'], 
                               aggfunc=np.max)

df_bki_result = df_bki_result[feature_high_importances]
#df_bki_result['APPLICATION_NUMBER'] = df_bki['APPLICATION_NUMBER']

In [ ]:
df_bki_result

# история платежей клиента;
df_payments

In [ ]:
#!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
# история платежей клиента;
df_payments = pd.read_csv(
    "../geekbrains-competitive-data-analysis/payments.csv"
)

print("payments.shape = {} rows, {} cols".format(*df_payments.shape))
df_payments.head()

In [ ]:
df_payments.info()

In [ ]:
numerical_features_df_payments = df_payments.select_dtypes(include=[np.number]).columns
numerical_features_df_payments = numerical_features_df_payments.drop('APPLICATION_NUMBER')
numerical_features_df_payments = numerical_features_df_payments.drop('PREV_APPLICATION_NUMBER')

numerical_features_df_payments

In [ ]:
df_payments = df_payments.fillna(0)

In [ ]:
df_payments_result = df_payments.copy()

df_payments_result = pd.pivot_table(df_payments_result,  
                               index=['APPLICATION_NUMBER'], 
                               aggfunc=np.median)

In [ ]:
df_payments_result.drop(['PREV_APPLICATION_NUMBER'], axis='columns', inplace=True)

In [ ]:
df_payments_result.head()

In [ ]:
# df_payments_scale = pd.DataFrame(standard_scaler.fit_transform(df_payments), 
#                columns=df_payments.columns)

# df_payments_scale.drop(['APPLICATION_NUMBER', 'PREV_APPLICATION_NUMBER'], axis='columns', inplace=True)

In [ ]:
# kmeans_payments = KMeans(n_clusters=10, max_iter=10000, random_state=1)

In [ ]:
# %%time

# kmeans_train_payments = kmeans_payments.fit_predict(df_payments_scale)

In [ ]:
# df_payments_result = pd.DataFrame(pd.get_dummies(kmeans_train_payments, prefix='payments'))
# df_payments_result_columns = df_payments_result.columns
# df_payments_result['APPLICATION_NUMBER'] = df_payments['APPLICATION_NUMBER']

In [ ]:
# df_payments_result = pd.pivot_table(df_payments_result, values=df_payments_result_columns, index=['APPLICATION_NUMBER'], aggfunc=np.sum)

In [ ]:
# df_payments_result['sum'] = df_payments_result.sum(axis=1)
# for col in df_payments_result_columns:
#     df_payments_result[col] = df_payments_result[col] / df_payments_result['sum']
# df_payments_result.drop(['sum'], axis='columns', inplace=True)  

# клиентский профиль, некоторые знания, которые есть у компании о клиенте;
df_client_profile

In [ ]:
# клиентский профиль, некоторые знания, которые есть у компании о клиенте;
df_client_profile = pd.read_csv(
    "../geekbrains-competitive-data-analysis/client_profile.csv"
)

print("client_profile.shape = {} rows, {} cols".format(*df_client_profile.shape))
df_client_profile.head()

In [ ]:
df_client_profile.info()

In [ ]:
msno.matrix(df_client_profile)

GENDER

In [ ]:
df_client_profile['GENDER'].value_counts()

In [ ]:
df_client_profile['GENDER'] = (df_client_profile['GENDER'] == 'F').astype(int)

FAMILY_STATUS,EDUCATION_LEVEL

In [ ]:
df_client_profile['FAMILY_STATUS'].value_counts()

In [ ]:
df_client_profile['EDUCATION_LEVEL'].value_counts()

In [ ]:
df_client_profile = create_freq_feature(df_client_profile, 'FAMILY_STATUS')
df_client_profile = create_freq_feature(df_client_profile, 'EDUCATION_LEVEL')

In [ ]:
df_client_profile = df_client_profile.fillna(value=
                                             {'OWN_CAR_AGE': 0, 
                                              'EXTERNAL_SCORING_RATING_1': df_client_profile['EXTERNAL_SCORING_RATING_1'].min(),
                                              'EXTERNAL_SCORING_RATING_3': df_client_profile['EXTERNAL_SCORING_RATING_3'].min()
                                             })

df_client_profile = df_client_profile.fillna(0)

In [ ]:
df_client_profile.describe().T

In [ ]:
df_train = pd.read_csv(
    "../geekbrains-competitive-data-analysis/train.csv"
)

In [ ]:
df_train.head()

In [ ]:
df_client_profile_t = df_client_profile.merge(df_train.loc[:, ['APPLICATION_NUMBER', 'TARGET']], 
                                            left_on='APPLICATION_NUMBER', 
                                            right_on='APPLICATION_NUMBER', 
                                            how='inner')


In [ ]:
df_client_profile_t

In [ ]:
%%time

dtrain = xgb.DMatrix(data=df_client_profile_t.iloc[:, :df_client_profile_t.shape[1]-1], 
                     label=df_client_profile_t.iloc[:, df_client_profile_t.shape[1]-1])
print('*')

params = {
    "booster": "gbtree",
    "objective": "binary:logistic",
    "eval_metric": "auc",
    "learning_rate": 0.01,
    "n_estimators": 500,
    "reg_lambda": 100,
    "max_depth": 10,
    "gamma": 10,
    "nthread": 6,
    "seed": 27
}

model_xgb = xgb.train(
    params=params,
    dtrain=dtrain,
    num_boost_round=1000,
    early_stopping_rounds=50,
    evals=[(dtrain, "train")],
    verbose_eval=100,
    maximize=True,
)

In [ ]:
x_valid_ = df_client_profile_t.sample(5000)
y_valid_ =  x_valid_['TARGET']
x_valid_.drop(['TARGET'], axis='columns', inplace=True)

In [ ]:
explainer = shap.TreeExplainer(model_xgb)
shap_values = explainer.shap_values(x_valid_, y_valid_)

In [ ]:
shap.summary_plot(shap_values, x_valid_, plot_type="bar", max_display=150)

In [ ]:
shap.summary_plot(shap_values, x_valid_, plot_type="dot")

In [ ]:
mean = list()

for j in range(shap_values.shape[1]):
    bufer = 0
    for i in range(shap_values.shape[0]):
        bufer += np.abs(shap_values[i][j])
    bufer /= shap_values.shape[0]
    mean.append(bufer)

mean = np.array(mean).reshape(-1, shap_values.shape[1])

feature_importances = pd.DataFrame(mean, columns=x_valid_.columns, index=['importances'])
feature_importances = feature_importances.sort_values('importances', axis=1, ascending=0)
max_num_feature = feature_importances.loc['importances', feature_importances.loc['importances',:]>0].shape[0]
feature_high_importances = feature_importances.iloc[: ,0:max_num_feature].columns
feature_high_importances

In [ ]:
%%time

df_train_, df_valid_ = train_test_split(df_client_profile_t, 
                                        test_size=0.4,  
                                        random_state=42)
df_valid_, df_test_ = train_test_split(df_valid_, 
                                        test_size=0.5,  
                                        random_state=42)

dtrain = lgb.Dataset(data=df_train_[feature_high_importances], 
                     label=df_train_['TARGET'])
dvalid = lgb.Dataset(data=df_valid_[feature_high_importances], 
                     label=df_valid_['TARGET'])

params = {
    "boosting_type": "goss",
    "objective": "binary",
    "metric": "auc",
    "learning_rate": 0.01,
    "n_estimators": 10000,
    "n_jobs": 6,
    "seed": 27
}

model_lgb = lgb.train(
    params=params,
    train_set=dtrain,
    num_boost_round=10000,
    valid_sets=[dtrain, dvalid],
    categorical_feature="auto",
    early_stopping_rounds=50,
    verbose_eval=100
)

#------------------------------------------------------------------------------
#lgb
from pylab import rcParams
rcParams['figure.figsize'] = 7, 7

pred = model_lgb.predict(df_test_[feature_high_importances])
fpr, tpr, _ = roc_curve(df_test_['TARGET'], pred)
roc_auc = auc(fpr, tpr)
plt.plot(fpr, tpr, label='xgb (area = %0.4f)' % roc_auc)

plt.plot([0, 1], [0, 1], color='navy', linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver operating characteristic example')
plt.legend(loc="lower right", prop={'size': 20})
plt.show()

In [ ]:
df_client_profile_result = df_client_profile[feature_high_importances]
df_client_profile_result['APPLICATION_NUMBER'] = df_client_profile['APPLICATION_NUMBER']

# Объединение

In [ ]:
df_train = pd.read_csv(
    "../geekbrains-competitive-data-analysis/train.csv"
)

df_test = pd.read_csv(
    "../geekbrains-competitive-data-analysis/test.csv"
)

print("gf_train.shape = {} rows, {} cols".format(*df_train.shape))
print("df_test.shape = {} rows, {} cols".format(*df_test.shape))

In [ ]:
#df_train = create_dummie(df_train)
#df_test = create_dummie(df_test)

df_train['NAME_CONTRACT_TYPE'] = (df_train['NAME_CONTRACT_TYPE'] == 'Cash').astype(int)
df_test['NAME_CONTRACT_TYPE'] = (df_test['NAME_CONTRACT_TYPE'] == 'Cash').astype(int)

In [ ]:
df_train, df_valid = train_test_split(df_train, 
                                      test_size=0.3,  
                                      random_state=42)

In [ ]:
print("gf_train.shape = {} rows, {} cols".format(*df_train.shape))
print("df_valid.shape = {} rows, {} cols".format(*df_valid.shape))
print("df_test.shape = {} rows, {} cols".format(*df_test.shape))

In [ ]:
df_train = df_train.merge(df_client_profile_result, left_on='APPLICATION_NUMBER', right_on='APPLICATION_NUMBER', how='left')
df_valid = df_valid.merge(df_client_profile_result, left_on='APPLICATION_NUMBER', right_on='APPLICATION_NUMBER', how='left')
df_test = df_test.merge(df_client_profile_result, left_on='APPLICATION_NUMBER', right_on='APPLICATION_NUMBER', how='left')

In [ ]:
df_train = df_train.merge(df_applications_history_result, left_on='APPLICATION_NUMBER', right_on='APPLICATION_NUMBER', how='left')
df_valid = df_valid.merge(df_applications_history_result, left_on='APPLICATION_NUMBER', right_on='APPLICATION_NUMBER', how='left')
df_test = df_test.merge(df_applications_history_result, left_on='APPLICATION_NUMBER', right_on='APPLICATION_NUMBER', how='left')

In [ ]:
df_train = df_train.merge(df_bki_result, left_on='APPLICATION_NUMBER', right_on='APPLICATION_NUMBER', how='left')
df_valid = df_valid.merge(df_bki_result, left_on='APPLICATION_NUMBER', right_on='APPLICATION_NUMBER', how='left')
df_test = df_test.merge(df_bki_result, left_on='APPLICATION_NUMBER', right_on='APPLICATION_NUMBER', how='left')

In [ ]:
df_train = df_train.merge(df_payments_result, left_on='APPLICATION_NUMBER', right_on='APPLICATION_NUMBER', how='left')
df_valid = df_valid.merge(df_payments_result, left_on='APPLICATION_NUMBER', right_on='APPLICATION_NUMBER', how='left')
df_test = df_test.merge(df_payments_result, left_on='APPLICATION_NUMBER', right_on='APPLICATION_NUMBER', how='left')

In [ ]:
cor = df_train.corr()
fig, axes = plt.subplots(figsize = (25,25))
plt.title("Correlation of Numeric Features with Sale Price", y=1, size=16)
sns.heatmap(cor, square=True, vmax=1, cmap="viridis")

In [ ]:
y_train = df_train["TARGET"]
y_valid = df_valid["TARGET"]

df_train.drop(["TARGET", 'APPLICATION_NUMBER'], axis='columns', inplace=True)
df_valid.drop(["TARGET", 'APPLICATION_NUMBER'], axis='columns', inplace=True)

In [ ]:
df_test.drop(['APPLICATION_NUMBER'], axis='columns', inplace=True)

In [ ]:
print("df_train.shape = {} rows, {} cols".format(*df_train.shape))
print("df_valid.shape = {} rows, {} cols".format(*df_valid.shape))
print("df_test.shape = {} rows, {} cols".format(*df_test.shape))

In [ ]:
df_train.describe().T

In [ ]:
for i in range(df_train.shape[1]):
    feature = df_train.iloc[:, i].name
    median = df_train.iloc[:, i].median()
    df_train.loc[df_train[feature].isnull(), [feature]]= median  

In [ ]:
msno.matrix(df_train)

In [ ]:
%%time

dtrain = xgb.DMatrix(data=df_train.fillna(-9999), label=y_train)
print('*')
dvalid = xgb.DMatrix(data=df_valid.fillna(-9999), label=y_valid)
print('*')
dtest = xgb.DMatrix(data=df_test.fillna(-9999))

In [ ]:
%%time

params = {
    "booster": "gbtree",
    "objective": "binary:logistic",
    "eval_metric": "auc",
    "learning_rate": 0.01,
    "n_estimators": 1000,
    "reg_lambda": 100,
    "max_depth": 10,
    "gamma": 10,
    "nthread": 6,
    "seed": 27
}

model_xgb = xgb.train(
    params=params,
    dtrain=dtrain,
    num_boost_round=1000,
    early_stopping_rounds=50,
    evals=[(dtrain, "train"), (dvalid, "valid")],
    verbose_eval=100,
    maximize=True,
)

In [ ]:
x_valid_ = df_valid.fillna(-9999).sample(5000)
y_valid_ =  y_valid.sample(5000)

explainer = shap.TreeExplainer(model_xgb)
shap_values = explainer.shap_values(x_valid_, y_valid_)

shap.summary_plot(shap_values, x_valid_, plot_type="bar", max_display=150)

In [ ]:
shap.summary_plot(shap_values, x_valid_)

In [ ]:
%%time

df_train_, df_valid_ = train_test_split(df_train, 
                                        test_size=0.4,  
                                        random_state=42)
df_valid_, df_test_ = train_test_split(df_valid_, 
                                        test_size=0.5,  
                                        random_state=42)

dtrain = lgb.Dataset(data=df_train_[feature_high_importances], 
                     label=df_train_['TARGET'])
dvalid = lgb.Dataset(data=df_valid_[feature_high_importances], 
                     label=df_valid_['TARGET'])

params = {
    "boosting_type": "goss",
    "objective": "binary",
    "metric": "auc",
    "learning_rate": 0.01,
    "n_estimators": 10000,
    "n_jobs": 6,
    "seed": 27
}

model_lgb = lgb.train(
    params=params,
    train_set=dtrain,
    num_boost_round=10000,
    valid_sets=[dtrain, dvalid],
    categorical_feature="auto",
    early_stopping_rounds=50,
    verbose_eval=100
)

#------------------------------------------------------------------------------
#lgb
from pylab import rcParams
rcParams['figure.figsize'] = 7, 7

pred = model_lgb.predict(df_test_[feature_high_importances])
fpr, tpr, _ = roc_curve(df_test_['TARGET'], pred)
roc_auc = auc(fpr, tpr)
plt.plot(fpr, tpr, label='xgb (area = %0.4f)' % roc_auc)

plt.plot([0, 1], [0, 1], color='navy', linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver operating characteristic example')
plt.legend(loc="lower right", prop={'size': 20})
plt.show()

In [ ]:
%%time

dtrain = lgb.Dataset(data=df_train.fillna(-9999), label=y_train)
dvalid = lgb.Dataset(data=df_valid.fillna(-9999), label=y_valid)
dtest = lgb.Dataset(data=df_test.fillna(-9999))

In [ ]:
%%time

params = {
    "boosting_type": "goss",
    "objective": "binary",
    "metric": "auc",
    "learning_rate": 0.01,
    "n_estimators": 5000,
    "n_jobs": 6,
    "seed": 27
}

model_lgb = lgb.train(
    params=params,
    train_set=dtrain,
    num_boost_round=10000,
    valid_sets=[dtrain, dvalid],
    categorical_feature="auto",
    early_stopping_rounds=50,
    verbose_eval=100
)

In [ ]:
result = model_lgb.predict(df_test.fillna(-9999))
result

In [ ]:
df_sample_submit["TARGET"] = df_sample_submit["TARGET"].astype('float')

In [ ]:
df_sample_submit["TARGET"] = result

In [ ]:
df_sample_submit.to_csv('result.csv', index=False) 

Только профиль

In [ ]:
df_train = pd.read_csv(
    "../geekbrains-competitive-data-analysis/train.csv"
)

df_test = pd.read_csv(
    "../geekbrains-competitive-data-analysis/test.csv"
)

print("gf_train.shape = {} rows, {} cols".format(*df_train.shape))
print("df_test.shape = {} rows, {} cols".format(*df_test.shape))

#df_train = create_dummie(df_train)
#df_test = create_dummie(df_test)

df_train['NAME_CONTRACT_TYPE'] = (df_train['NAME_CONTRACT_TYPE'] == 'Cash').astype(int)
df_test['NAME_CONTRACT_TYPE'] = (df_test['NAME_CONTRACT_TYPE'] == 'Cash').astype(int)

In [ ]:
df_train = df_train.merge(df_client_profile_result, left_on='APPLICATION_NUMBER', right_on='APPLICATION_NUMBER', how='left')
df_test = df_test.merge(df_client_profile_result, left_on='APPLICATION_NUMBER', right_on='APPLICATION_NUMBER', how='left')

In [ ]:
msno.matrix(df_train)

In [ ]:
df_train.loc[df_train['AGE'].isnull(), 'TARGET']

In [ ]:
plt.figure(figsize=(8,5))

sns.countplot(x='TARGET',data=df_train.loc[df_train['AGE'].isnull(), :])
plt.title('Target variable distribution')

plt.show()

In [ ]:
df_train = pd.read_csv(
    "../geekbrains-competitive-data-analysis/train.csv"
)

df_test = pd.read_csv(
    "../geekbrains-competitive-data-analysis/test.csv"
)

print("gf_train.shape = {} rows, {} cols".format(*df_train.shape))
print("df_test.shape = {} rows, {} cols".format(*df_test.shape))

#df_train = create_dummie(df_train)
#df_test = create_dummie(df_test)

df_train['NAME_CONTRACT_TYPE'] = (df_train['NAME_CONTRACT_TYPE'] == 'Cash').astype(int)
df_test['NAME_CONTRACT_TYPE'] = (df_test['NAME_CONTRACT_TYPE'] == 'Cash').astype(int)

df_train, df_valid = train_test_split(df_train, 
                                      test_size=0.3,  
                                      random_state=42)




In [ ]:
df_train = df_train.merge(df_client_profile_result, left_on='APPLICATION_NUMBER', right_on='APPLICATION_NUMBER', how='left')
df_valid = df_valid.merge(df_client_profile_result, left_on='APPLICATION_NUMBER', right_on='APPLICATION_NUMBER', how='left')
df_test = df_test.merge(df_client_profile_result, left_on='APPLICATION_NUMBER', right_on='APPLICATION_NUMBER', how='left')

In [ ]:
y_train = df_train["TARGET"]
y_valid = df_valid["TARGET"]

df_train.drop(["TARGET", 'APPLICATION_NUMBER'], axis='columns', inplace=True)
df_valid.drop(["TARGET", 'APPLICATION_NUMBER'], axis='columns', inplace=True)
df_test.drop(['APPLICATION_NUMBER'], axis='columns', inplace=True)

In [ ]:
for i in range(df_train.shape[1]):
    feature = df_train.iloc[:, i].name
    median = df_train.iloc[:, i].mean()
    df_train.loc[df_train[feature].isnull(), [feature]]= median  

In [ ]:
%%time

dtrain = xgb.DMatrix(data=df_train, label=y_train)
print('*')
dvalid = xgb.DMatrix(data=df_valid, label=y_valid)
print('*')
dtest = xgb.DMatrix(data=df_test)

params = {
    "booster": "gbtree",
    "objective": "binary:logistic",
    "eval_metric": "auc",
    "learning_rate": 0.01,
    "n_estimators": 500,
    "reg_lambda": 100,
    "max_depth": 10,
    "gamma": 10,
    "nthread": 6,
    "seed": 27
}

model_xgb = xgb.train(
    params=params,
    dtrain=dtrain,
    num_boost_round=1000,
    early_stopping_rounds=50,
    evals=[(dtrain, "train"), (dvalid, "valid")],
    verbose_eval=100,
    maximize=True,
)

x_valid_ = df_valid.sample(5000)
y_valid_ =  y_valid

explainer = shap.TreeExplainer(model_xgb)
shap_values = explainer.shap_values(x_valid_, y_valid_)

shap.summary_plot(shap_values, x_valid_, plot_type="bar", max_display=150)
shap.summary_plot(shap_values, x_valid_,)